In [5]:
from PIL import Image;
import numpy as np;
import scipy as sci;
import csv;
import cv2;
import matplotlib.pyplot as plt;
import pandas as pd;
import os
from coins_utils import *

import time

def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
inputDir = 'CandidateDataSet2/augmented_and_resized_RAW/1cent/'
input_file = [inputDir+i for i in os.listdir(inputDir) if '.jpg' in i]
#input_file = glob.glob(inputDir+"*.jpg")
#input_file = [inputDir+"P70210-161658_Candidate_1_Aug_100_0.jpg"]  #For debugging

outputDir = 'CandidateDataSet2/augmented_and_resized_brightness_yuv_RAW/1cent/' #CHANGE IT!
brightness = [-25, 0, 25]
ROWS = 100
COLS = 100

for kfile in input_file:
    for kbright in brightness:
        tic()
        #print("Augmenting image "+str(kfile))
        im = Image.open(kfile) #Open image
        imarray = np.array(im)
        imarray = cv2.resize(imarray, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)
        #imarray = cv2.erode(imarray, None, 1) 
        yuv = cv2.cvtColor(imarray, cv2.COLOR_BGR2YUV) #Convert it to yuv
        y, u, v = cv2.split(yuv)
        y = y.astype(np.int16)
        y += kbright
        y = np.maximum(np.zeros(y.shape),y)
        y = np.minimum(255*np.ones(y.shape),y)
        y = y.astype(np.uint8)
        final_yuv = cv2.merge((y, u, v))
        img_bright = cv2.cvtColor(final_yuv, cv2.COLOR_YUV2BGR)
        #plt.imshow(img_bright)
        #plt.show()
        
        #fileNamek = kfile.split("\\")[1]
        fileNamek = kfile.split("/")[-1]
        output_filename = outputDir+fileNamek.split(".")[0]+"_Brightness_"+str(kbright)+".jpg"
        img = Image.fromarray(img_bright, 'RGB')
        img.save(output_filename)
        #print(output_filename)
    #cv2.imwrite("image_processed.jpg", img)
    
    
    #for kAngle in angles:
    #    M = cv2.getRotationMatrix2D((imarray.shape[1]/2,imarray.shape[0]/2),kAngle,1)
    #    rotated_image = cv2.warpAffine(imarray,M,(imarray.shape[1],imarray.shape[0]))
    #    plt.imshow(rotated_image)
    #    plt.show()
    #    fileNamek = kfile.split("\\")[1]
    #    output_filename = outputDir+fileNamek.split(".")[0]+"_Aug_"+str(kAngle)+".jpg"
    #    img = Image.fromarray(rotated_image, 'RGB')
    #    img.save(output_filename)
print("Data augmentation with brightness completed!")    

Data augmentation with brightness completed!


# Use data_ls to create the labels!